## Getting started with Spark dataframes.

To start working with Spark dataframes, we first need to start a Spark session.

In [1]:
import findspark

In [2]:
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('haveFun').getOrCreate()

In [5]:
df1 = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [6]:
df1

DataFrame[age: bigint, name: string]

In [8]:
df1.show()
# Nothings takes place till we apply an 'action'. 

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [9]:
df1.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



The above function prints the schema of the table. It tells that there are two columns -- age and name and their type.

In [10]:
df1.columns

['age', 'name']

In [11]:
df1.describe()

DataFrame[summary: string, age: string, name: string]

In [12]:
df1.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



describe: statistical summary of numeric data columns.

### Import type tools to handle schema properly. 

In [13]:
from pyspark.sql.types import (StructField, IntegerType, 
                              StringType, StructType)

Now create a list of StructFields. StructFields take in three parameters -- name, datatype, and if nulllable.

In [18]:
data_schema = [StructField('age', IntegerType(), True), 
              StructField('name', StringType(), True)]
# the second argument is class instance. 
# the third fields tells if null values are ok or not.
# This is a schema that we are expecting.

In [19]:
final_struc = StructType(fields=data_schema)

In [20]:
df2 = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json', schema = final_struc)

In [21]:
df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Extracting columns

In [22]:
df2['age']

Column<b'age'>

In [23]:
type(df2['age'])

pyspark.sql.column.Column

df2['age'] gives a column object.

In [25]:
df2.select('age')

DataFrame[age: int]

In [26]:
df2.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [28]:
type(df2.select('age'))

pyspark.sql.dataframe.DataFrame

In [37]:
# Selecting multiple columns
df2.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



df2.select gives a dataframe.

Extracting rows

In [30]:
df2.head(2)
# gives a list of rows

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [31]:
df2.head(2)[1]

Row(age=30, name='Andy')

In [35]:
type(df2.head())
# df2.head() gives a row object. 

pyspark.sql.types.Row

So far we have seen a column object, a row object, and of course a dataframe object. Specialized objects allow distributed computing.  

#### Adding new columns

In [41]:
df2.withColumn('doubleAge', 2.0 * df2['age']).show()

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|     60.0|
|  19| Justin|     38.0|
+----+-------+---------+



In [43]:
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



withColumn is not an inplace operation. 

We can also rename a column. 

In [45]:
df2.withColumnRenamed('age', 'newAge').show()

+------+-------+
|newAge|   name|
+------+-------+
|  null|Michael|
|    30|   Andy|
|    19| Justin|
+------+-------+



In [48]:
df2.show()
# withColumnRenamed is also not an inplace operation.

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Using pure SQL on Spark.

In [49]:
df2.createOrReplaceTempView('people')

In [50]:
results = spark.sql("SELECT * FROM people")

In [51]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [57]:
results2 = spark.sql("SELECT * FROM people WHERE name = 'Justin'")

In [58]:
results2.show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+

